In [6]:
import sys
sys.executable
sys.path

['c:\\gpu_env\\scripts\\python36.zip',
 'c:\\gpu_env\\DLLs',
 'c:\\gpu_env\\lib',
 'c:\\gpu_env\\scripts',
 'C:\\Program Files\\Python36\\Lib',
 'C:\\Program Files\\Python36\\DLLs',
 'c:\\gpu_env',
 '',
 'c:\\gpu_env\\lib\\site-packages',
 'c:\\gpu_env\\lib\\site-packages\\IPython\\extensions',
 'C:\\Users\\hsl\\.ipython']

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts

# load data

In [2]:
    tr_005930 = pd.read_hdf(r'\data\tickdata\20190426_kor_stock.h5', "005930_tr")
    tr_000660 = pd.read_hdf(r'\data\tickdata\20190426_kor_stock.h5', "000660_tr")
    tr_012330 = pd.read_hdf(r'\data\tickdata\20190426_kor_stock.h5', "012330_tr")
    tr_005380 = pd.read_hdf(r'\data\tickdata\20190426_kor_stock.h5', "005380_tr")

    cur_005930 = tr_005930.reset_index().drop(columns =['time_api']).set_index(['datetime_sys'])
    cur_000660 = tr_000660.reset_index().drop(columns =['time_api']).set_index(['datetime_sys'])
    cur_012330 = tr_012330.reset_index().drop(columns =['time_api']).set_index(['datetime_sys'])
    cur_005380 = tr_005380.reset_index().drop(columns =['time_api']).set_index(['datetime_sys'])

    cur_1m_005930 = cur_005930['price'].resample('1T').ohlc().dropna()
    cur_1m_000660 = cur_000660['price'].resample('1T').ohlc().dropna()
    cur_1m_012330 = cur_012330['price'].resample('1T').ohlc().dropna()
    cur_1m_005380 = cur_005380['price'].resample('1T').ohlc().dropna()

    close_005930 = cur_1m_005930['close'].to_frame()
    close_000660 = cur_1m_000660['close'].to_frame()
    close_012330 = cur_1m_012330['close'].to_frame()
    close_005380 = cur_1m_005380['close'].to_frame()

    close_005930.columns =['A005930']
    close_000660.columns =['A000660']
    close_012330.columns =['A012330']
    close_005380.columns =['A005380']

    preset_005930 = close_005930.iloc[:-100, :]
    preset_000660 = close_000660.iloc[:-100, :]
    preset_005380 = close_005380.iloc[:-100, :]
    preset_012330 = close_012330.iloc[:-100, :]

    test_005930 = close_005930.iloc[-100:, :]
    test_000660 = close_000660.iloc[-100:, :]
    test_005380 = close_005380.iloc[-100:, :]
    test_012330 = close_012330.iloc[-100:, :]

    test_data = pd.concat([test_005930, test_000660, test_005380, test_012330], axis=1)

    symbols =[close_005930,
                close_000660,
                close_005380,
                close_012330]

    combined_symbols = pd.concat(symbols, join = 'outer', axis=1)
    combined_symbols = combined_symbols.fillna(method = 'ffill')
    combined_symbols = combined_symbols.dropna()
    # p = pairs(close_005930, close_000660)
    # p.cointegration_test()
    # p.adf
    #
    # p.num_bar


    training_period = combined_symbols.index

In [7]:
training_period

DatetimeIndex(['2019-04-26 09:00:00', '2019-04-26 09:01:00',
               '2019-04-26 09:02:00', '2019-04-26 09:03:00',
               '2019-04-26 09:04:00', '2019-04-26 09:05:00',
               '2019-04-26 09:06:00', '2019-04-26 09:07:00',
               '2019-04-26 09:08:00', '2019-04-26 09:09:00',
               ...
               '2019-04-26 15:17:00', '2019-04-26 15:18:00',
               '2019-04-26 15:19:00', '2019-04-26 15:30:00',
               '2019-04-26 15:40:00', '2019-04-26 15:41:00',
               '2019-04-26 15:42:00', '2019-04-26 15:43:00',
               '2019-04-26 15:44:00', '2019-04-26 15:45:00'],
              dtype='datetime64[ns]', name='datetime_sys', length=387, freq=None)

In [ ]:
class pairs(object):
    def __init__(self, a, b):
        self.a = a
        self.b = b
        self.name = a.name + ':' + b.name
        
        self.df = pd.concat([a, b], axis=1).dropna()
        # The number of bars in the rolling window would be determined by the resolution, so we get this
        #   information from the shape of the DataFrame here.
        # num_bar: entire time series
        self.num_bar = self.df.shape[0]

        # select iloc[0, 1] to since its 2x2 corr matrix
        self.cor = self.df.corr().iloc[0, 1]
        # Set the initial signals to be 0
        self.error = 0
        self.last_error = 0
        self.pos_a = 0
        self.pos_b = 0


    def cor_update(self):
        self.cor = self.df.corr().iloc[0, 1]

    def cointegration_test(self):
        # self.model = sm.OLS(formula = '%s ~ %s'%(str(self.a),str(self.b)), data = self.df).fit()

        constant_added = sm.add_constant(self.df.iloc[:, 1])
        self.model = sm.OLS(self.df.iloc[:, 0], constant_added).fit()
        # This line conduct ADF test on the residual. ts.adfuller() returns a tuple and the first element in
        #   the tuple is the test value.
        self.adf = ts.adfuller(self.model.resid, autolag='BIC')[0]
        self.mean_resid = np.mean(self.model.resid)
        self.sd = np.std(self.model.resid)
        print('pairs head: ', self.df.head())
        print('pairs tail: ', self.df.tail())
        print('adf: ', self.adf)
        print('resid mean: ', self.mean_resid)
        print('resid sd: ', self.sd)

    def df_update(self, a_new, b_new):
        # new_df = pd.DataFrame({str(self.a.columns): self.a_price, str(self.b): self.b}, index=[self.a.index]).dropna()
        new_df = pd.concat([a_new, b_new], axis=1)
        self.df = pd.concat([self.df, new_df], axis=0)
        #rolling windows to compute adf
        self.df = self.df.tail(self.num_bar)

In [ ]:
def generate_pairs(combined_price):
    # self.pair_num = num_pair
    pair_list = []

    for i in range(len(combined_price.columns)):
        for j in range(i + 1, len(combined_price.columns)):
            pair_list.append(pairs(combined_price.iloc[:, i], combined_price.iloc[:, j]))

    [print('inital pair: ', x.name, x.cor) for x in pair_list]

    pair_list = [x for x in pair_list if x.cor > pair_threshold]
    pair_list.sort(key=lambda x: x.cor, reverse=True)
    return pair_list

In [ ]:
def update_pairs(data, selected_final):
    num_select = len(selected_final)
    for pair in selected_final:
        # when selected pairs are in given data
        if pair.a.name in data.columns and pair.b.name in data.columns:
            pair.df_update(data.loc[:, pair.a.name], data.loc[:, pair.b.name])
        else:
            selected_final.remove(pair)

    for pair in selected_final:
        pair.last_error = pair.error

In [ ]:
def select_final_pairs(combined_price):

    if len(combined_price.index) == num_bar:  # self.count == 0 and
        # if self.generate_count == 0:
        # for symbol in self.symbols:
            # symbol.df = pd.DataFrame(symbol, index = symbol.index, columns = ['%s'%str(symbol)])
        pair_list = generate_pairs(combined_price)
            # self.generate_count += 1
            # self.Log('pair list length:'+str(len(self.pair_list)))

        # TODO: why do we need this? - initially inside generate pairs we already have corr
        # for pair in self.pair_list:
        #     pair.cor_update()

    # Update the DataFrame and correlation selection
    # TODO: why do we need this? - a_price can't be 0
    # if len(self.pair_list[0].a_price) != 0:
    #     for pair in self.pair_list:
    #         pair.df_update()
    #         pair.cor_update()

    # TODO: why do we double select pairs?? generate pairs have corr hurdle inside it
    # self.selected_corr = [x for x in self.pair_list if x.cor > 0.9]
    # Cointegration test
    for pair in pair_list:
        pair.cointegration_test()

    print('pair len: ', len(pair_list))
    print('df: ', pair_list[0].df.head(), pair_list[0].df.tail())
    print('adf: ', pair_list[0].adf)
    # print('bic: ', self.pair_list[0])
    # TODO: x.adf should be compared with BIC
    selected_adf = [x for x in pair_list if x.adf < -1]
    selected_adf.sort(key=lambda x: x.adf)
    # If no pair passed the two-stage test, return.


    if len(selected_adf) == 0:
        # self.Log('no selected pair')
        # TODO: why do we count when no pairs passed adf threshold?
        # self.count += 1
        return print('no selected pair')

    # clean the pair to avoid overlapping stocks.
    selected_final = pair_clean(selected_adf)
    # assign a property to the selected pair, this is a signal that would be used for trading.
    for pair in selected_final:
        pair.state = 0
        # self.Log(str(pair.adf) + pair.name)
    # limit the number of selected pairs.
    if len(selected_final) > selected_num:
        selected_final = selected_final[:selected_num]

    # self.count += 1
    # self.data_count = 0
    return


In [ ]:
    def open_pairs(selected_final):
        for pair in selected_final:

            # here error is updated first from initial 0
            # error is the residual from cointegration
            # what does error mean here, greater better? smaller better??? smaller better but small spread
            pair.error = pair.a.values[-1]- (pair.model.params[0] + pair.model.params[1] * pair.b.values[-1])


            # compare previous signal and current signal
            # signal meaning: in previous, signal under mean +sd and now its above it
            # -> spread recover??
            if pair.error > pair.mean_resid - pair.sd and pair.last_error < pair.mean_resid - pair.sd:
                # self.Log('long %s and short %s'%(str(i.a),str(i.b)))
                # why record model??
                pair.record_model = pair.model
                pair.record_mean_resid = pair.mean_resid
                pair.record_sd = pair.sd
                trading_pairs.append(pair)
                pair.pos_a = 1.0
                pair.pos_b = -1.0
                # self.positions['a'] = 5.0 / len(self.selected_final)  # self.SetHoldings(i.a, 5.0/(len(self.selected_final)))
                # self.positions['b'] = -5.0 / len(self.selected_final)  # self.SetHoldings(i.b, -5.0/(len(self.selected_final)))
                pair.state = 1

            # signal meaning: previously signal was above the mean +sd and now its below it
            # -> spread brought back
            elif pair.error < pair.mean_resid + pair.sd and pair.last_error > pair.mean_resid + pair.sd:
                # self.Log('long %s and short %s'%(str(i.b),str(i.a)))
                pair.record_model = pair.model
                pair.record_mean_resid = pair.mean_resid
                pair.record_sd = pair.sd
                trading_pairs.append(pair)

                pair.pos_a = -1.0
                pair.pos_b = 1.0
                # self.positions['b'] = 5.0 / len(self.selected_final)  # self.SetHoldings(i.b, 5.0/(len(self.selected_pair)))
                # self.positions['a'] = -5.0 / len(self.selected_final)  # self.SetHoldings(i.a, -5.0/(len(self.selected_pair)))
                pair.state = -1

            else:
                pair.state = 0

In [ ]:
# closing pairs
def close_pairs(data, trading_pairs):
    for i in trading_pairs:
        price_a = float(data[i.a].Close)
        price_b = float(data[i.b].Close)
        i.error = price_a - (i.record_model.params[0] + i.record_model.params[1] * price_b)

        # why multiply close_size or stop loss to sd??
        if i.error < i.record_mean_error + close_size * i.record_sd and i.last_error > i.record_mean_error + close_size * i.record_sd:
            return
        elif i.error > i.record_mean_error - close_size * i.record_sd and i.last_error < i.record_mean_error +  stop_loss * i.record_sd:
            return


        trading_pairs.remove(i)

In [ ]:
    pt.select_final_pairs()

    pt.update_pairs(test_data)
    pt.open_pairs()

    
combined_price = combined_symbols
num_bar = len(training_period) # num_bar: entire time series
        # self.positions = {}
selected_num = 2
pair_threshold = 0.4
trading_pairs = []